In [5]:
# !unzip -uq "TESTSET.zip" -d "TESTSET"

In [1]:
import sys
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten,  GlobalAveragePooling2D
from keras.layers.convolutional import Conv2D, MaxPooling2D
import numpy as np

import cv2
import os
import glob

/home/ubuntu/anaconda3/envs/amazonei_tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ubuntu/anaconda3/envs/amazonei_tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ubuntu/anaconda3/envs/amazonei_tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 

In [4]:
path_zero='dataset_5m/dataset_5m/E01' # 무표정
path_plus='dataset_5m/dataset_5m/E02' # 웃음
path_minus='dataset_5m/dataset_5m/E03' # 찡그림
# path_unkno='image/unknown'

la=len(os.listdir(path_zero))
lb=len(os.listdir(path_plus))
lc=len(os.listdir(path_minus))
# lc=len(os.listdir(path_unkno)) 

print('zero 경로에 저장된 파일의 개수:'+str(la))
print('plus 경로에 저장된 파일의 개수:'+str(lb))
print('minus 경로에 저장된 파일의 개수:'+str(lc))

zero 경로에 저장된 파일의 개수:14400
plus 경로에 저장된 파일의 개수:14400
minus 경로에 저장된 파일의 개수:14400


In [5]:
xsize=100
ysize=100

In [6]:
x=[]
y=[]

from keras.preprocessing import image
import numpy as np

def read_dir(path, label):
    
    files = glob.glob(path + "/*.jpg")
    if bool(files) == False:
        files = glob.glob(path + "/*.png") 
     
    for f in files:

        try:
            img = image.load_img(f, target_size=(xsize,ysize))
            img_tr= image.img_to_array(img)        
            img_tr /= 255.
            
            y.append(label)
            x.append(img_tr)
        except:
            pass

In [7]:
read_dir(path_zero, 0) 
read_dir(path_plus, 1) 
read_dir(path_minus, 2) 

In [8]:
x=np.array(x)
y=np.array(y)

In [9]:
print(x.shape)

(43200, 100, 100, 3)


In [10]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, stratify=y)

input_shape = (xsize,ysize, 3)

# batch_size = 32
# num_classes = 2
# epochs = 15

In [11]:
print(x_test.shape)
print(y_test.shape)

(4320, 100, 100, 3)
(4320,)


In [12]:
y_train = keras.utils.to_categorical(y_train)
y_test = keras.utils.to_categorical(y_test)

In [15]:
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
from keras.applications.xception import Xception
from keras.applications.mobilenet_v2 import MobileNetV2 


input_shape = (100, 100, 3)
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=input_shape)

# 미세 조정
base_model.trainable=True

set_trainable=False
for layer in base_model.layers:
    if layer.name == 'out_relu': # block8~9
        set_trainable=True
    if set_trainable:
        layer.trainable=True
    else:
        layer.trainable=False

x = Flatten()(base_model.output)
dropout = Dropout(0.98)
output = Dense(3, activation='softmax', name='predictions')(x)
model = Model(inputs=base_model.input, output=output)

model.compile(loss='categorical_crossentropy',optimizer=optimizers.RMSprop(lr=0.0001),metrics=['accuracy'])

model.summary()

/home/ubuntu/anaconda3/envs/amazonei_tensorflow_p36/lib/python3.6/site-packages/keras_applications/mobilenet_v2.py:294: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 100, 100, 3)  0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 101, 101, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 50, 50, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 50, 50, 32)   128         Conv1[0][0]                      
____________________________________________________________________________________________

/home/ubuntu/anaconda3/envs/amazonei_tensorflow_p36/lib/python3.6/site-packages/ipykernel_launcher.py:25: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("pr...)`


In [16]:
history = model.fit(x_train, y_train, validation_split=0.5, 
          epochs=30, batch_size=128, verbose=1) # DenseNet에 비해 속도가 빠름

acc = model.evaluate(x_test, y_test)
print(acc)

Train on 19440 samples, validate on 19440 samples
Epoch 1/30
19440/19440 [==============================] - 17s 859us/step - loss: 1.2189 - accuracy: 0.4837 - val_loss: 1.6361 - val_accuracy: 0.4379
Epoch 2/30
19440/19440 [==============================] - 15s 779us/step - loss: 0.9317 - accuracy: 0.5997 - val_loss: 1.5057 - val_accuracy: 0.4759
Epoch 3/30
19440/19440 [==============================] - 15s 781us/step - loss: 0.8250 - accuracy: 0.6454 - val_loss: 1.5860 - val_accuracy: 0.4563
Epoch 4/30
19440/19440 [==============================] - 15s 783us/step - loss: 0.7503 - accuracy: 0.6768 - val_loss: 1.7868 - val_accuracy: 0.4434
Epoch 5/30
19440/19440 [==============================] - 15s 784us/step - loss: 0.6909 - accuracy: 0.7044 - val_loss: 1.5168 - val_accuracy: 0.4849
Epoch 6/30
19440/19440 [==============================] - 15s 785us/step - loss: 0.6535 - accuracy: 0.7211 - val_loss: 1.8913 - val_accuracy: 0.4477
Epoch 7/30
19440/19440 [==============================] 

KeyboardInterrupt: 

In [25]:
# model.save('save/MobileNetV2_fine_tuning(full).h5')
# print('모델 저장이 완료되었습니다')

모델 저장이 완료되었습니다


In [ ]:
test_path = "dataset_pred/E01" # 무표정
test_path2 = "dataset_pred/E02" # 기쁨 
test_path3 = "dataset_pred/E03" # 찡그림

X=[]

def read_dir1(path):
    
    files = glob.glob(path + "/*.jpg")
    if bool(files) == False:
        files = glob.glob(path + "/*.png") 
     
    for f in files:

        try:
            img = image.load_img(f, target_size=(xsize,ysize))
            img_tr= image.img_to_array(img)        
            img_tr /= 255.
            X.append(img_tr)
        except:
            pass

In [ ]:
# test set 기쁨
read_dir1(test_path)
X = np.array(X)
pred = model.predict(X)

In [ ]:
# test set(무표정)
read_dir1(test_path2)
X = np.array(X)
pred = model.predict(X)

In [28]:
# test set(무표정)
read_dir1(test_path)
X = np.array(X)
pred = model.predict(X)

# 예측(test set 무표정)
co=0
for i in range(2880):
    print(np.argmax(pred[i]),end='')
    tmp=np.argmax(pred[i])
    
    if tmp == 0:
        co+=1

print('\n', "%.2f" % (co/2880.))

00000000000000000000000000001000000000000000000000000000000000000000000000000000000000000020000000000000000000000000000000000000000000000000002000000000000000000000000000000000000000000000000000000000
 0.005


In [30]:
# test set(기쁨)
read_dir1(test_path2)
X = np.array(X)
pred = model.predict(X)

# 예측(test set 기쁨)
co=0
for i in range(2880):
    print(np.argmax(pred[i]),end='')
    tmp=np.argmax(pred[i])
    
    if tmp == 1:
        co+=1

print('\n', "%.2f" % (co/2880.))

00000000000000000000000000001000000000000000000000000000000000000000000000000000000000000020000000000000000000000000000000000000000000000000002000000000000000000000000000000000000000000000000000000000
 0.985


In [ ]:
# test set(찡그림)
read_dir1(test_path3)
X = np.array(X)
pred = model.predict(X)

# 예측(test set 찡그림)
co=0
for i in range(2880):
    print(np.argmax(pred[i]),end='')
    tmp=np.argmax(pred[i])
    
    if tmp == 2:
        co+=1

print('\n', "%.2f" % (co/2880.))

In [0]:
# 기쁨 정확도 : 0 %
# 무표정 정확도 : 80 % 

# 역시 무표정 판별률이 더 높다!


# 기쁨 정확도 : 5%
# 무표정 정확도 : 98.5% 